In [391]:
import pandas as pd 
import numpy as np
from datetime import datetime
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from publicsuffix2 import get_public_suffix

In [432]:
import pandas 
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [433]:
# def ohe_mail_type (df):
#     unique_mail_types = df['mail_type'].apply(lambda x: x.split('/')).explode().unique()
#     for mt in unique_mail_types:
#         df[f'mail_type_{mt}'] = df['mail_type'].apply(lambda x: 1 if mt in x else 0)
#     df.drop('mail_type',axis =1,inplace = True)
#     return(df)


# def data_cleaner (df, chars_in_subject = 42 ):
#     df.chars_in_subject = df.chars_in_subject.fillna(10)
#     df['org'] = df['org'].fillna('org_undefinded')
#     df['tld'] = df['tld'].fillna('tld_undefinded')
#     df['mail_type'] = df['mail_type'].fillna('mail_type_undefinded').apply(lambda x:x.lower())
#     df['chars_in_subject'] = df['chars_in_subject'].fillna(chars_in_subject)
#     df.drop([ 'date','tld'],axis= 1, inplace=True)
#     return (df)

In [434]:
train_df = pd.read_csv('train.csv', )
test_df = pd.read_csv('test.csv', )
# train_df.drop('Unnamed: 0',axis=1,inplace = True)
# test_df.drop('Unnamed: 0',axis=1,inplace = True)
test_df['label'] = 'test_df'
df = train_df.append(test_df, ignore_index=True)
# df[df ['Unnamed: 0'] == 40]

C:\Users\arind\AppData\Local\Temp\ipykernel_25892\1600230955.py:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = train_df.append(test_df, ignore_index=True)


In [435]:
df['chars_in_subject'] = df['chars_in_subject'].fillna(10)
df.org = df.org.fillna('org_unknown')
df['Ratio-SubtoBody'] = df['chars_in_body'] / (df['chars_in_subject'] + df['chars_in_body'])

In [436]:
df['tld'] = df['tld'].fillna('notfound.com').apply(get_public_suffix)

C:\Users\arind\anaconda3\lib\site-packages\publicsuffix2\__init__.py:376: UserWarning: This function returns the private suffix, SLD, or registrable domain. This equivalent to function get_sld(). To get the public suffix itself, use get_tld().
  warnings.warn(


In [437]:
import datefinder
# df['day'] = df['date'].apply(lambda x:x.split(',')[0])
def find_date(x):
    for x in datefinder.find_dates(x):
        return(x)
        break

df['date'] = df['date'].apply(find_date)

In [438]:
df['date'] = pd.to_datetime(df['date'], utc = True)
df['year'] = pd.DatetimeIndex(df['date']).year
df['month'] = pd.DatetimeIndex(df['date']).month
df['weekday'] = pd.DatetimeIndex(df['date']).dayofweek
df['hour'] = pd.DatetimeIndex(df['date']).hour
df['monthday'] = pd.DatetimeIndex(df['date']).dayofyear
df.drop('date',axis=1,inplace=True)

In [439]:
df['tld'] = df['tld'].fillna('com')
df['tld_back'] = df['tld'].apply(lambda x : x.split('.')[-1] if '.' in x else x ).apply(lambda x : x.lower())
df['mail_type'] = df['mail_type'].apply(lambda x : str(x).lower()).fillna('multipart/alternative')


In [440]:
df = pd.get_dummies(df, prefix=['MailType'], columns = ['mail_type'],drop_first=False)

In [441]:
df['tld_front'] = df.tld.apply(lambda x: x.split('.')[0] if '.' in x else '0')

In [442]:
a = df.dtypes.to_frame('type').reset_index()
a[(a['type']=='object') ]['index'].to_list()

['org', 'tld', 'label', 'tld_back', 'tld_front']

In [443]:
df.tld

0              ebay.in
1         efinmail.com
2                  com
3                   ai
4                  com
              ...     
114536             com
114537    sdconnect.in
114538           ac.in
114539             com
114540    sdconnect.in
Name: tld, Length: 114541, dtype: object

In [444]:
for x in a[(a['type']=='object') ]['index'].to_list(): 
    if x != 'label':
        le_org = preprocessing.LabelEncoder()
        df[x] = le_org.fit_transform(df[x])

In [445]:

df = df.set_index('Unnamed: 0', drop=True)



In [446]:
to_scale = []
for col in df.columns:
    if len(df[col].unique())>2:
        to_scale.append(col)
 

In [447]:
to_scale

['org',
 'tld',
 'ccs',
 'images',
 'urls',
 'chars_in_subject',
 'chars_in_body',
 'label',
 'Ratio-SubtoBody',
 'year',
 'month',
 'weekday',
 'hour',
 'monthday',
 'tld_back',
 'tld_front']

In [448]:
to_scale_stsc = [
 'images',
 'urls',
 'chars_in_subject',
 'chars_in_body','Ratio-SubtoBody']

In [449]:
to_scale_nosc = list(set(to_scale) - set(to_scale_stsc))

In [450]:
from sklearn.preprocessing import StandardScaler
for val in to_scale_stsc:
    sc = StandardScaler()
#     df[val] = sc.fit_transform(df[val])
    df[val] = sc.fit_transform(df[[val]])
# sc.transform

In [456]:
for val in to_scale_nosc:
    
#     df[val] = sc.fit_transform(df[val])
    if val!= 'label':
        df[val] = (df[val] - df[val].mean())/df[val].max()

In [458]:
to_scale_nosc

['label',
 'hour',
 'org',
 'year',
 'tld_front',
 'tld_back',
 'tld',
 'monthday',
 'weekday',
 'ccs',
 'month']

In [459]:
train_df = df[df['label'] != 'test_df']
train_df['label'] = train_df['label'].astype('int')
test_df = df[df['label'] == 'test_df']

C:\Users\arind\AppData\Local\Temp\ipykernel_25892\2717079273.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['label'] = train_df['label'].astype('int')


In [460]:
train_df

,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,...,MailType_multipart/mixed,MailType_multipart/related,MailType_multipart/report,MailType_multipart/signed,MailType_nan,MailType_text/calendar,MailType_text/html,MailType_text/html,MailType_text/plain,tld_front
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0.216331,0.020119,-0.002673,0,0.058459,0.727489,0,0,-0.073733,-0.062080,...,0,0,0,0,0,0,0,0,0,0.170998
1,-0.233762,0.033678,-0.002673,0,-0.018750,-0.235281,0,0,1.645460,-0.102522,...,0,0,0,0,0,0,0,0,0,0.183709
2,0.399325,-0.037508,-0.002673,0,-0.011938,-0.150080,0,0,-0.488711,-0.090878,...,0,0,0,0,0,0,1,0,0,-0.095952
3,-0.120083,-0.230728,-0.002673,0,-0.021021,-0.294922,0,0,-1.081536,0.229309,...,1,0,0,0,0,0,0,0,0,-0.095952
4,-0.137643,-0.037508,0.002914,0,-0.016480,-0.201201,0,0,-0.073733,-0.102840,...,0,0,0,0,0,0,0,0,0,-0.095952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
80171,-0.137643,-0.037508,0.002914,0,-0.021021,-0.226761,0,0,1.171200,-0.103390,...,0,0,0,0,0,0,0,0,0,-0.095952
80172,0.132227,0.389611,-0.002673,0,-0.021021,-0.277882,1,0,0.637657,-0.097010,...,0,0,0,0,0,0,1,0,0,0.531167
80173,-0.127477,-0.037508,-0.002673,0,0.040292,0.429286,0,0,-0.162657,-0.089128,...,0,0,0,0,0,0,0,0,0,-0.095952


In [415]:
import numpy as np

from scipy.stats import uniform, randint

from sklearn.datasets import load_breast_cancer, load_diabetes, load_wine
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV, train_test_split

import xgboost as xgb

In [416]:
X_train, X_test, y_train, y_test = train_test_split(
    train_df.drop('label',axis=1), train_df.label, test_size=0.1, random_state=42)

In [418]:
import xgboost

model = xgboost.XGBRFClassifier(n_estimators=100, subsample=0.9, colsample_bynode=0.2)

In [426]:
train_df.columns

Index(['org', 'tld', 'ccs', 'bcced', 'images', 'urls', 'salutations',
       'designation', 'chars_in_subject', 'chars_in_body', 'label',
       'Ratio-SubtoBody', 'year', 'month', 'weekday', 'hour', 'monthday',
       'tld_back', 'MailType_multipart/alternative', 'MailType_multipart/idm',
       'MailType_multipart/mixed', 'MailType_multipart/related',
       'MailType_multipart/report', 'MailType_multipart/signed',
       'MailType_nan', 'MailType_text/calendar', 'MailType_text/html',
       'MailType_text/html ', 'MailType_text/plain', 'tld_front'],
      dtype='object')

In [420]:
pred = model.predict(X_test)

In [422]:
accuracy_score(y_test,pred)

0.5452731354452482

In [461]:
X_train, X_test, y_train, y_test = train_test_split(
    train_df.drop('label',axis=1), pd.get_dummies(train_df.label), test_size=0.1, random_state=42)

In [462]:
model = Sequential()
model.add(Dense(29, input_shape=(29,), activation='relu'))
# model.add(Dense(29, activation='relu'))
model.add(Dense(29, activation='relu'))
model.add(Dense(8, activation='sigmoid'))

In [463]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(X_train,y_train,validation_data=(X_test, y_test), epochs=5, batch_size=10)

Epoch 1/5
7216/7216 [==============================] - 6s 751us/step - loss: 0.2384 - accuracy: 0.4883 - val_loss: 0.2226 - val_accuracy: 0.5084
Epoch 2/5
7216/7216 [==============================] - 5s 740us/step - loss: 0.2191 - accuracy: 0.5090 - val_loss: 0.2148 - val_accuracy: 0.5157
Epoch 3/5
7216/7216 [==============================] - 5s 742us/step - loss: 0.2143 - accuracy: 0.5224 - val_loss: 0.2107 - val_accuracy: 0.5308
Epoch 4/5
7216/7216 [==============================] - 5s 745us/step - loss: 0.2112 - accuracy: 0.5304 - val_loss: 0.2090 - val_accuracy: 0.5337
Epoch 5/5
7216/7216 [==============================] - 5s 742us/step - loss: 0.2090 - accuracy: 0.5353 - val_loss: 0.2085 - val_accuracy: 0.5378


In [211]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Activation, Embedding, Flatten, LeakyReLU, BatchNormalization, Dropout
from keras.activations import relu, sigmoid
from keras.layers import LeakyReLU

from sklearn.preprocessing import StandardScaler

In [212]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [213]:
def create_model(layers, activation):
    model = Sequential()
    for i, nodes in enumerate(layers):
        if i==0:
            model.add(Dense(nodes,input_dim=X_train.shape[1]))
            model.add(Activation(activation))
        else:
            model.add(Dense(nodes))
            model.add(Activation(activation))
    model.add(Dense(8)) # Note: no activation beyond this point
    
    model.compile(optimizer='adam', loss='binary_crossentropy',metrics=['accuracy'])
    return model
    
model = KerasClassifier(build_fn=create_model, verbose=0)

C:\Users\arind\AppData\Local\Temp\ipykernel_25892\3610109041.py:15: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KerasClassifier(build_fn=create_model, verbose=0)


In [214]:
layers = [[20], [40, 20], [45, 30, 15]]
activations = ['sigmoid', 'relu']
param_grid = dict(layers=layers, activation=activations, batch_size = [128, 256], epochs=[30])
grid = GridSearchCV(estimator=model, param_grid=param_grid)

In [464]:
df

,org,tld,ccs,bcced,images,urls,salutations,designation,chars_in_subject,chars_in_body,...,MailType_multipart/mixed,MailType_multipart/related,MailType_multipart/report,MailType_multipart/signed,MailType_nan,MailType_text/calendar,MailType_text/html,MailType_text/html,MailType_text/plain,tld_front
Unnamed: 0,,,,,,,,,,,,,,,,,,,,,
0,0.216331,0.020119,-0.002673,0,0.058459,0.727489,0,0,-0.073733,-0.062080,...,0,0,0,0,0,0,0,0,0,0.170998
1,-0.233762,0.033678,-0.002673,0,-0.018750,-0.235281,0,0,1.645460,-0.102522,...,0,0,0,0,0,0,0,0,0,0.183709
2,0.399325,-0.037508,-0.002673,0,-0.011938,-0.150080,0,0,-0.488711,-0.090878,...,0,0,0,0,0,0,1,0,0,-0.095952
3,-0.120083,-0.230728,-0.002673,0,-0.021021,-0.294922,0,0,-1.081536,0.229309,...,1,0,0,0,0,0,0,0,0,-0.095952
4,-0.137643,-0.037508,0.002914,0,-0.016480,-0.201201,0,0,-0.073733,-0.102840,...,0,0,0,0,0,0,0,0,0,-0.095952
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34360,0.190453,-0.037508,-0.002673,0,-0.002854,0.642288,1,1,2.445774,-0.057335,...,0,0,0,0,0,0,0,0,0,-0.095952
34361,0.258845,0.515034,-0.002673,0,0.031209,0.701929,1,0,0.015190,-0.091355,...,0,0,0,0,0,0,0,0,0,0.662523
34362,-0.077569,-0.254457,-0.002673,0,-0.018750,-0.260841,1,0,-0.488711,-0.101684,...,0,0,0,0,0,0,0,0,0,-0.066291
